<a href="https://colab.research.google.com/github/DonghaeSuh/NLP_Pytorch/blob/main/Model/Seq2Seq_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 간단한 전처리
- 특수 문자 제거
- 특수 토큰 추가 MAX_LENGTH인 25에 맞춰서 시작 토큰과 끝 토큰을 추가 -> 이후 field에서 추가해서 사용
- decoder의 입력에 따른 예측값에 해당하는 dataset을 마련

In [ ]:
import pandas as pd

In [ ]:
ls

drive/  sample_data/


In [ ]:
cd drive/MyDrive/Pytorch\ NLP/data_in/Chatbot

/content/drive/MyDrive/Pytorch NLP/data_in/Chatbot


In [ ]:
ls

ChatbotData.csv  Mecab-ko-for-Google-Colab/


In [ ]:
df=pd.read_csv('ChatbotData.csv')
df.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


### 특수 문자 제거

In [ ]:
df['Q']=df['Q'].str.replace('[^ a-zA-Z0-9가-힣]',"")
df['A']=df['A'].str.replace('[^ a-zA-Z0-9가-힣]',"")
df.head()

<ipython-input-90-21d281052103>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Q']=df['Q'].str.replace('[^ a-zA-Z0-9가-힣]',"")
<ipython-input-90-21d281052103>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['A']=df['A'].str.replace('[^ a-zA-Z0-9가-힣]',"")


,Q,A,label
0,12시 땡,하루가 또 가네요,0
1,1지망 학교 떨어졌어,위로해 드립니다,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠,0
4,PPL 심하네,눈살이 찌푸려지죠,0


In [ ]:
def decoder_input(sent,Max_len):
  sent=sent.split()
  if len(sent)>Max_len-2: # Max_length=25
    sent=sent[:Max_len-2]
    sent.append('<eos>')
    sent[:0]=['<sos>']
  else:
    sent.append('<eos>')
    sent[:0]=['<sos>']
  
  return " ".join(sent)

In [ ]:
def decoder_output(sent):
  sent=sent.split()
  sent.pop(0)
  return " ".join(sent)

In [ ]:
# 알고보니 torchtext의 필드정의에서 <sos>, <eos> token 추가를 지원했다. => 이럴 경우 굳이 미리 추가해놓고 준배해줄 필요가 없다.
df['label']=df['A']
temp=pd.Series(df['Q'].tolist()+df['A'].tolist()).astype(str)
temp.to_csv('vocab_word.csv',index=False)
df.head()

,Q,A,label
0,12시 땡,하루가 또 가네요,하루가 또 가네요
1,1지망 학교 떨어졌어,위로해 드립니다,위로해 드립니다
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠,여행은 언제나 좋죠
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠,여행은 언제나 좋죠
4,PPL 심하네,눈살이 찌푸려지죠,눈살이 찌푸려지죠


In [ ]:
df.to_csv('2_Fine_ChatbotData.csv',index=False)

In [ ]:
decoder_input('안녕하세요 저는 사람입니다',4)

'<sos> 안녕하세요 저는 <eos>'

In [ ]:
decoder_output('<sos> 안녕하세요 저는 <eos>')

'안녕하세요 저는 <eos>'

In [ ]:
df['A']=df['A'].apply(lambda x : decoder_input(x,25))
df.head()

,Q,A,label
0,12시 땡,<sos> 하루가 또 가네요 <eos>,0
1,1지망 학교 떨어졌어,<sos> 위로해 드립니다 <eos>,0
2,3박4일 놀러가고 싶다,<sos> 여행은 언제나 좋죠 <eos>,0
3,3박4일 정도 놀러가고 싶다,<sos> 여행은 언제나 좋죠 <eos>,0
4,PPL 심하네,<sos> 눈살이 찌푸려지죠 <eos>,0


In [ ]:
df['label']=df['A'].apply(decoder_output)
df.head()

,Q,A,label
0,12시 땡,<sos> 하루가 또 가네요 <eos>,하루가 또 가네요 <eos>
1,1지망 학교 떨어졌어,<sos> 위로해 드립니다 <eos>,위로해 드립니다 <eos>
2,3박4일 놀러가고 싶다,<sos> 여행은 언제나 좋죠 <eos>,여행은 언제나 좋죠 <eos>
3,3박4일 정도 놀러가고 싶다,<sos> 여행은 언제나 좋죠 <eos>,여행은 언제나 좋죠 <eos>
4,PPL 심하네,<sos> 눈살이 찌푸려지죠 <eos>,눈살이 찌푸려지죠 <eos>


In [ ]:
df.to_csv('Fine_ChatbotData.csv',index=False)

# 데이터 불러오기 및 dataset,dataLoader 생성

In [1]:
import pandas as pd

In [2]:
ls

drive/  sample_data/


In [3]:
cd drive/MyDrive/Pytorch\ NLP/data_in/Chatbot

/content/drive/MyDrive/Pytorch NLP/data_in/Chatbot


In [4]:
ls

2_Fine_ChatbotData.csv  Fine_ChatbotData.csv        vocab_word.csv
ChatbotData.csv         Mecab-ko-for-Google-Colab/


In [5]:
vocab_word=pd.read_csv('vocab_word.csv')
vocab_word.head()

,0
0,12시 땡
1,1지망 학교 떨어졌어
2,3박4일 놀러가고 싶다
3,3박4일 정도 놀러가고 싶다
4,PPL 심하네


In [6]:
train_data=pd.read_csv('2_Fine_ChatbotData.csv')
train_data.head()

,Q,A,label
0,12시 땡,하루가 또 가네요,하루가 또 가네요
1,1지망 학교 떨어졌어,위로해 드립니다,위로해 드립니다
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠,여행은 언제나 좋죠
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠,여행은 언제나 좋죠
4,PPL 심하네,눈살이 찌푸려지죠,눈살이 찌푸려지죠


tokenizer = 공백기준분리 str.split() 사용 예정


In [7]:
## hyperparameter

BATCH_SIZE=2
MAX_LEN=25
EPOCH=50
UNIT=1024
EMB_DIM=256

### 필드 정의 및 Tabulardataset 생성

In [8]:
pip install torchtext==0.6

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.1 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.1
    Uninstalling torchtext-0.15.1:
      Successfully uninstalled torchtext-0.15.1


In [9]:
from torchtext import data

In [10]:
# vocab 전용
V=data.Field(sequential=True,
             use_vocab=True,
             tokenize=str.split,
             init_token='<sos>',
             eos_token='<eos>',
             fix_length=MAX_LEN)

In [11]:
Q=data.Field(sequential=True,
             use_vocab=True,
             tokenize=str.split,
             batch_first=True,
             fix_length=MAX_LEN)

A=data.Field(sequential=True,
             use_vocab=True,
             init_token='<sos>',
             eos_token='<eos>',
             tokenize=str.split,
             batch_first=True,
             fix_length=MAX_LEN)
LABEL=data.Field(sequential=True,
                 use_vocab=True,
                 eos_token='<eos>',
                 tokenize=str.split,
                 batch_first=True,
                 fix_length=MAX_LEN)

In [12]:
from torchtext.data import TabularDataset

In [13]:
train_data=TabularDataset('2_Fine_ChatbotData.csv',format='csv',fields=[('question',Q),('answer',A),('label',LABEL)],skip_header=True)
vars(train_data[0])

{'question': ['12시', '땡'],
 'answer': ['하루가', '또', '가네요'],
 'label': ['하루가', '또', '가네요']}

In [14]:
vocab_data=TabularDataset('vocab_word.csv',format='csv',fields=[('vocab_word',V)],skip_header=True)

### 사전 정의
- 3개의 Field는 하나의 사전을 공유
- vocab_word라는 Q와A의 모든 문장을 합친 데이터로부터 단어사전을 만든 다음에
- 해당 단어사전을 Q,A,LABEL에 전달할 예정

In [15]:
V.build_vocab(vocab_data,min_freq=5)

Q.vocab=V.vocab
A.vocab=V.vocab
LABEL.vocab=V.vocab

In [ ]:
Q.vocab.stoi

In [16]:
Q.vocab.itos[943]

'안녕'

In [17]:
Q.vocab.stoi['안녕']

943

### 시드 고정 후 데이터 분리

In [18]:
import random
import torch
SEED=123
random.seed(SEED)
torch.manual_seed(SEED)

train_data,val_data=train_data.split(split_ratio=0.9,random_state=random.seed(SEED))

### 데이터 로더 생성

In [19]:
from torchtext.data import Iterator

In [20]:
train_iter=Iterator(train_data,batch_size=BATCH_SIZE)
val_iter=Iterator(val_data,batch_size=BATCH_SIZE)

In [21]:
batch=next(iter(train_iter))
batch.question

tensor([[ 742,  202, 1854,    0,  211,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1],
        [  45,    0,    0, 2612,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1]])

In [22]:
batch.answer

tensor([[   2,    0,    0,    0,  858,  321,    0,   89,    3,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1],
        [   2,  978, 1768,  539,   89,    3,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1]])

In [23]:
batch.label

tensor([[   0,    0,    0,  858,  321,    0,   89,    3,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1],
        [ 978, 1768,  539,   89,    3,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1]])

In [24]:
train_iter=Iterator(train_data,batch_size=BATCH_SIZE)
val_iter=Iterator(val_data,batch_size=BATCH_SIZE)

# 모델 구현

In [25]:
## hyperparameter

BATCH_SIZE=2
MAX_LEN=25
EPOCH=50
UNIT=1024
EMB_DIM=256

In [26]:
## 모듈 불러오기
import torch
import torch.nn as nn

In [27]:
class Encoder(nn.Module):
  def __init__(self,vocab_size,emb_dim,hidden_dim,dense_dim):
    super(Encoder,self).__init__()
    self.embedding=nn.Embedding(vocab_size,emb_dim)
    self.gru=nn.GRU(emb_dim,hidden_dim,batch_first=True)
    self.dense=nn.Linear(hidden_dim,dense_dim)

  def forward(self,x):
    x=self.embedding(x)  # (batch_size, Max_len, emb_dim)
    outputs,last_hidden_state=self.gru(x) # outputs: ( batch_size, Max_len, dense_dim ) last_hidden_state : ( batch_size, hidden_dim )
    
    return self.dense(outputs),self.dense(last_hidden_state) # outputs: ( batch_size, Max_len, dense_dim ) last_hidden_state : ( batch_size, hidden_dim )

In [28]:
class Attention(nn.Module):
  def __init__(self,dense_dim):
    super(Attention,self)
    self.dense=nn.Linear(dense_dim,1)
    self.softmax=nn.Softmax(dim=1)

  def forward(self,outputs,last_hidden_state):
    after_sum=outputs+last_hidden_state  # ( batch_size, Max_len, dense_dim )
    after_dense=self.dense(after_sum) # ( batch_size, Max_len, 1 )
    after_softmax=self.softmax(after_dense) # ( batch_size, Max_len, 1 )
    context_vector=torch.sum(after_softmax*outputs,dim=1) # ( batch_size, dense_dim )

    return context_vector # ( batch_size, dense_dim )

In [ ]:
## Sum
a=torch.Tensor([[[1,2,3],[4,5,6]]])
b=torch.Tensor([[1,2,3]])
print(a.size())
print(b.size())
a+b

torch.Size([1, 2, 3])
torch.Size([1, 3])


tensor([[[2., 4., 6.],
         [5., 7., 9.]]])

In [ ]:
## Multi token Sum
a=torch.Tensor([[[1,2,3],[4,5,6]]])
b=torch.Tensor([[[1,2,3]],[[4,5,6]]])
print(a.size())
print(b.size())
a+b

torch.Size([1, 2, 3])
torch.Size([2, 1, 3])


tensor([[[ 2.,  4.,  6.],
         [ 5.,  7.,  9.]],

        [[ 5.,  7.,  9.],
         [ 8., 10., 12.]]])

In [ ]:
## Softmax
softmax=nn.Softmax(dim=1)
a=torch.Tensor([[[3],[5],[4],[1]]])
out=softmax(a)
out

tensor([[[0.0889],
         [0.6572],
         [0.2418],
         [0.0120]]])

In [ ]:
## Weighted Sum
a=torch.Tensor([[[1,2,3],[4,5,6]]])
b=torch.Tensor([[[1],[2]]])
print(a.size())
print(b.size())
a*b

torch.Size([1, 2, 3])
torch.Size([1, 2, 1])


tensor([[[ 1.,  2.,  3.],
         [ 8., 10., 12.]]])

In [30]:
class Decoder(nn.Module):
  def __init__(self,vocab_size,emb_dim,hidden_dim,dense_dim):
    super(Decoder,self).__init__()
    self.embedding=nn.Embedding(vocab_size,emb_dim)
    self.gru=nn.GRU(emb_dim+dense_dim,hidden_dim,batch_first=True)
    self.dense=nn.Linear(hidden_dim,vocab_size)
  
  def forward(self,x,context):
    # x : ( batch_size, k )    / k : 현재 Deocder에서 처리하고 있는 토큰의 index+1 # index=[0,max_length]
    # context : ( bath_size, k, dense_dim==emb_dim)
    after_emb=self.embedding(x) # ( batch_size, k , emb_dim)
    after_cat=torch.cat([context,after_emb],dim=-1)  # ( batch_size, k , emb_dim+dense_dim)
    output,state=self.gru(after_cat) # ( batch_size, k , hidden_dim), ( batch_size, hidden_dim)
    predict_token=self.dense(output) # ( batch_size, k, hidden_dim) -> ( batch_size, k, vocab_size )
    return  predict_token, state # (batch_size, k, vocab_size) , ( batch_size, hidden_dim )

In [ ]:
a=torch.Tensor([[[1,2,3],[4,5,6]],[[3,2,1],[6,5,4]]])
b=torch.Tensor([[[1,2,3],[4,5,6]],[[4,5,6],[7,8,9]]])
print(a.size())
print(b.size())
torch.cat([b,a],dim=-1)

torch.Size([2, 2, 3])
torch.Size([2, 2, 3])


tensor([[[1., 2., 3., 1., 2., 3.],
         [4., 5., 6., 4., 5., 6.]],

        [[4., 5., 6., 3., 2., 1.],
         [7., 8., 9., 6., 5., 4.]]])

In [43]:
a=torch.Tensor([[[1,2,3]],[[6,5,4]]])
b=torch.Tensor([[[1,2,3]],[[4,5,6]]])
print(a.size(),a,sep='\n')
print(b.size(),b,sep='\n')
torch.cat([a,b],dim=1)

torch.Size([2, 1, 3])
tensor([[[1., 2., 3.]],

        [[6., 5., 4.]]])
torch.Size([2, 1, 3])
tensor([[[1., 2., 3.]],

        [[4., 5., 6.]]])


tensor([[[1., 2., 3.],
         [1., 2., 3.]],

        [[6., 5., 4.],
         [4., 5., 6.]]])

In [71]:
class Seq2seq(nn.Module):
  def __init__(self,vocab_size,emb_dim,hidden_dim,dense_dim):
    super(Seq2seq,self).__init__()
    self.encoder=Encoder(vocab_size,emb_dim,hidden_dim,dense_dim)
    self.attention=Attention(dense_dim)
    self.decoder=Decoder(vocab_size,emb_dim,hidden_dim,dense_dim)

  def forward(self,enc_input,dec_input): # enc_input : ( batch_size, max_len ) , dec_input : ( batch_size, max_ len )
    # outputs: ( batch_size, Max_len, dense_dim ) last_hidden_state : ( batch_size, hidden_dim )
    outputs,last_hidden_state=self.encoder(enc_input)
    context=self.attention(outputs,last_hidden_state) # ( batch_size, dense_dim )
    context=torch.unsqueeze(context,dim=1) # ( batch_size, 1, dense_dim ) <= 여기에다가 25개의 context_vector들을 모을 것임

    for i in range(1,MAX_LEN+1):
      predict,state=self.decoder(dec_input[:,:i],context)  # (batch_size, i, vocab_size) , ( batch_size, hidden_dim )
      new_context=self.attention(outputs,state) # ( batch_size, dense_dim )
      new_context=torch.unsqueeze(new_context,dim=1) # ( batch_size, 1, dense_dim )
      context=torch.cat([context,new_context],dim=1) # ( batch_size, i+1, dense_dim ) <= 다음 loop에서 context로 사용
    
    # 이렇게 돌고 맨 마지막으로 나온 25개의 연결된 값을 반환
    return predict # ( batch_size, max_len==25, vocab_size )
    # -> 이에 대한 label : ( batch_size, max_len ) : crossenctorpyloss 사용하여 각각의 label에 해당하는 index(해당 단어의 index)의 요소의 값 가장 크도록 학습

In [49]:
a=torch.Tensor([[1,2,3],[4,5,6]])
a=torch.unsqueeze(a,dim=1)
print(a.size(),a,sep='\n')

torch.Size([2, 1, 3])
tensor([[[1., 2., 3.]],

        [[4., 5., 6.]]])


In [69]:
a=torch.Tensor([[1,2,3],[4,5,6]])
a.size()
a[:,:2]

tensor([[1., 2.],
        [4., 5.]])